In [2]:
from PyPDF4 import PdfFileReader
import textdistance as td
import re

In [3]:
reader = PdfFileReader(open('tax-guide.pdf', 'rb'))

In [4]:
content = ""
indexes = []
for i in range(0,1898):
    cur_page = reader.getPage(i).extractText()
    content += cur_page + '\n'
    
    if len(re.split(r'\nA\. (\w| )+\n', cur_page)) > 1:
        indexes.append(i+1)
        
content = content.split('ey.com/GlobalTaxGuides\n \ney.com/TaxGuidesApp\n')

In [34]:
country_dict = {}
page_no = {}
for i in range(1, len(content)):
    country_dict[content[i-1].split('\n')[-2]] = content[i]
    page_no[content[i-1].split('\n')[-2]] = indexes[i-1]

In [32]:
rates_dict = {}
for key in country_dict.keys():
                    
    country_rates = (re.split(r'\n[A-F]\. (\w| )+\n',
                              (re.sub(r'\([a-z]\)', '', country_dict[key])))[2]
                       .split('\n \n'))
    
    for i in reversed(range(len(country_rates))):

        if country_rates[i][:1] == ' ':
            j = 1
            while not re.match(r'[A-Z]', country_rates[i-j][:1]):
                j += 1
            country_rates[i] = country_rates[i-j] + 'on ' + country_rates[i][2:]
    
    rates_dict[key] = {}
    for i in range(len(country_rates)):
        if country_rates[i].replace('\n', '').isnumeric():
            rates_dict[key][country_rates[i-1].replace('\n', '').strip()] = country_rates[i].replace('\n', '').strip()

['Corporate Income Tax Rate (%)', '20', 'Capital Gains Tax Rate (%)', '20', 'Business Receipts Tax Rate (%)', '4/5/10', '', 'Withholding Tax (%) ', 'Withholding Tax (%) on Dividends', '20', '', 'Withholding Tax (%) on Interest', '20', '', 'Withholding Tax (%) on Royalties from Patents, Know-how, etc.', '20', '', 'Withholding Tax (%) on Commissions', '20', '', 'Net Operating Losses (Years)', 'Net Operating Losses (Years)on Carryback', '0', 'Net Operating Losses (Years)on Carryforward', '3', '\n', 'This tax is imposed on total gross revenue before deductions. It is a deduct\n-\nible expense in computing taxable income.\n', 'This withholding tax is considered a final settlement of the tax liability.\n', 'Losses can be generally carried forward in equal proportions to each of the \nfollowing three years. Unrestricted loss carryforwards are allowed for speci\n-\nfied companies.']
['Corporate Profits Tax Rate (%)', '15', 'Capital Gains Tax Rate (%)', '15', 'Branch Tax Rate (%)', '15', 'Withh

In [106]:
def fuzzy_match(list1, list2):
    min_dist = 10
    min_word1 = ''
    min_word2 = ''
    
    for word1 in list1:
        for word2 in list2:
            cur_dist = td.levenshtein.distance(word1.replace('(%)', '').lower(), 
                                               word2.replace('(%)', '').lower())
            if cur_dist < min_dist:
                min_dist = cur_dist
                min_word1 = word1
                min_word2 = word2
                
    return [min_word1, min_word2]

In [92]:
def get_subsentences(string):
    sentence = string.split(' ')
    return [' '.join(sentence[i:j]) for i in range(len(sentence)) 
            for j in range(i + 1, len(sentence) + 1)]

In [145]:
def get_answer(question):
    substrings = get_subsentences(question)
    country = fuzzy_match(substrings, rates_dict.keys())[1]
    rate = fuzzy_match(substrings, rates_dict[country].keys())[1]
    
    if rate.endswith('(%)'):
        return ('The ' + rate[:-4] + ' for ' + country + ' is ' + rates_dict[country][rate] + '%. ' +
                'Check page ' + str(page_no[country]) + ' of the tax guide for more information.')
    else:
        return ('The ' + rate + ' for ' + country + ' is ' + rates_dict[country][rate] + '. '
                'Check page ' + str(page_no[country]) + ' of the tax guide for more information.')

In [158]:
get_answer("astralia branch tax rate")

'The Branch Tax Rate for Australia is 30%. Check page 88 of the tax guide for more information.'